In [76]:
import re
from random import uniform
from collections import defaultdict

r_alphabet = re.compile(u'[а-яА-Я0-9-]+|[.,:;?!]+')

def gen_lines(corpus):
    data = open(corpus)
    for line in data:
        yield line.lower()

def gen_tokens(lines):
    for line in lines:
        for token in r_alphabet.findall(line):
            yield token

def gen_trigrams(tokens):
    t0, t1 = '$', '$'
    for t2 in tokens:
        yield t0, t1, t2
        if t2 in '.!?':
            yield t1, t2, '$'
            yield t2, '$','$'
            t0, t1 = '$', '$'
        else:
            t0, t1 = t1, t2

def train(corpus):
    lines = gen_lines(corpus)
    tokens = gen_tokens(lines)
    trigrams = gen_trigrams(tokens)

    bi, tri = defaultdict(lambda: 0.0), defaultdict(lambda: 0.0)

    for t0, t1, t2 in trigrams:
        bi[t0, t1] += 1
        tri[t0, t1, t2] += 1

    model = {}
    for (t0, t1, t2), freq in tri.items():
        if (t0, t1) in model:
            model[t0, t1].append((t2, freq/bi[t0, t1]))
        else:
            model[t0, t1] = [(t2, freq/bi[t0, t1])]
    return model

def generate_sentence(model):
    phrase = ''
    t0, t1 = '$', '$'
    while 1:
        t0, t1 = t1, unirand(model[t0, t1])
        if t1 == '$': break
        if t1 in ('.!?,;:') or t0 == '$':
            phrase += t1
        else:
            phrase += ' ' + t1
    return phrase.capitalize()

def unirand(seq):
    sum_, freq_ = 0, 0
    for item, freq in seq:
        sum_ += freq
    rnd = uniform(0, sum_)
    for token, freq in seq:
        freq_ += freq
        if rnd < freq_:
            return token


In [94]:
def generate_sentence_based(model, first_word='', n=100):
    phrase = ''
    if (len(first_word)>0):
        t0, t1 = '$', first_word
    else:
        t0, t1 = '$', '$'
    while n>0:
        n-=1
        t0, (t1, fr) = t1, unirand_freq(model[t0, t1])
        if t1 == '$': break
        if t1 in ('.!?,;:') or t0 == '$':
            phrase += t1 
        else:
            phrase += ' ' + t1
    return first_word.capitalize()+''+ phrase.capitalize()

In [95]:
def unirand_freq(seq):
    sum_, freq_ = 0, 0
    for item, freq in seq:
        sum_ += freq
    rnd = uniform(0, sum_)
    for token, freq in seq:
        freq_ += freq
        if rnd < freq_:
            return (token, freq)

In [96]:
for i in range(10):
    print (generate_sentence(model))

-- о, и все так делают.
Он все читал, отмечая сотни чертой мела на воротах.
Как будто облегчение от своих сомнений и вследствие этого недоброжелательно.
Она не знала.
-- я так и текут.
Она здесь еще третью часть своих средств на устройство, сооружение франц.
--!
;,!
Он что-то слишком горячо заступался за дубкова, у тебя лишний работник пропал -- как вам сказать, - быть понятым - выражается в одном - позволь, люди этого войска не позволял этого.
- да отчего ж не увидимся.


In [97]:
model = train('tolstoy.txt')

In [98]:
for i in range(10):
    print (generate_sentence_based(model, 'тогда'))

Тогда только про то, что, потому что попросил меня разбить завивку, и даже любовь, про которую говорили, что власть есть власть есть власть, если на это из канавы к лошадям.
Тогда эта встреча не произвела на него испуганными глазами подошел к стулу, чтоб образовать людей твердых, добродетельных матери и, чтобы они последние перешли и зажгли мост, сбрасывает мешки с горючими веществами в воду и выпивали ее до тех пор как приехал николай, что кампания кончена?
Тогда ему казалось, стояли пешие.
Тогда, когда у противника шестнадцать шашек, а мы все ездили к ним повозки: человек в сером сюртуке встал и подошел к столу, достал тетрадь, в том, что, прости врагам моим!
Тогда на пути жертвованья она могла требовать этого и взять ее на прежнее место.
Тогда эта встреча не произвела на него взглядов докторов, обыкновенно говоривший легко и хорошо.
Тогда я сказал: - ты будешь или дипломат?
Тогда не пришли к безусловной свободе вне пространства, что без малейшего промежутка, сопровождается таким огл

In [5]:
import pandas as pd

In [6]:
df1 = pd.read_csv('24.csv')
df2 =pd.read_csv('lenta.csv')
df3 = pd.read_csv('vesti.csv')
df = pd.concat([df1,df2,df3])
del df1
del df2
del df3
df = df.dropna()

In [19]:
text = ' \n '.join(df.text)

In [22]:
file1 = open("MyFile.txt","w") 
file1.write(text)
file1.close() 

In [23]:
model = train('MyFile.txt')
for i in range(10):
    print (generate_sentence(model))

Следующее евровидение пройдет с 14 по словам специалистов, прибор регистрации параметров полета подвергся интенсивному температурному воздействию и получил свою демократию, потому что после избрания главой правительства чечни в 2015 году украина и 1, забил 32 мяча и отдал ему свое сердце он завещал похоронить в польше, германии, раскритиковав политику украинизации страны.
Мама ирина всегда полностью соблюдала все рекомендации по порядку возбуждения дел, возбужденных по статьям хулиганство и побои.
К 20 дням ареста за вторжение в германию в 1882 году он перенес несколько сердечных приступов.
Преподавателя по музыке задержали правоохранители.
В настоящее время под наблюдением врачей находятся 62 человека, выяснить, чем дольше ребенок находится в деликатной ситуации: есть опасения нехватки сырья, а от того, что изучает размер ущерба, а фронтальный 1920х1080 точек 30 60, а также оказать помощь при госпитализации подъяпольского.
Семь процентов россиян.
Для детей старше 16 лет.
.
Пострадавши